In [19]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

# 1. Load and normalize CIFAR10

In [20]:
transform= transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

In [21]:
batch_size = 4

In [22]:
trainset = torchvision.datasets.CIFAR10(root='./data',train=True,
                                       download=True, transform=transform)

Files already downloaded and verified


In [23]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True)

In [24]:
len(trainset) 

50000

In [25]:
len(trainloader)

12500

In [26]:
testset = torchvision.datasets.CIFAR10(root='./data',train=False,
                                       download=True, transform=transform)

Files already downloaded and verified


In [27]:
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

In [28]:
len(testloader)

2500

# 2. Define a Convolutional Neural Network

In [40]:
#without using sequential layer
class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        self.conv1= nn.Conv2d(3,6,5)
        self.bn1=nn.BatchNorm2d(6)
        self.conv2= nn.Conv2d(6,16,5)
        self.bn2=nn.BatchNorm2d(16)
        
        self.fc1= nn.Linear(16*5*5, 120) # 5*5 from image dimension
        self.bn3=nn.BatchNorm1d(120)
        self.fc2= nn.Linear(120,84)
        self.bn4=nn.BatchNorm1d(84)
        self.fc3= nn.Linear(84,10)
        
    def forward(self, input):
        x=self.conv1(input)
        x=F.relu(x)
        x=F.max_pool2d(x,(2,2))
        x=self.bn1(x)
        x=self.conv2(x)
        x=F.relu(x)
        x=F.max_pool2d(x,(2,2))
        x=self.bn2(x)
        
        x= torch.flatten(x,1)
        x=self.fc1(x)
        x=F.relu(x)
        x=self.bn3(x)
        x=self.fc2(x)
        x=F.relu(x)
        x=self.bn4(x)
        x=self.fc3(x)
        return x
        
        

In [30]:
model= Net()
print(model)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (bn3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (bn4): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [41]:
#using sequential layer
#conv/FC -> Batchnorm -> activation func -> Pooling -> dropuout -> FC
class Net1(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.layers= nn.Sequential(
        nn.Conv2d(3,6,5), 
        nn.BatchNorm2d(6),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        
        nn.Conv2d(6,16,5),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
            
        nn.Flatten(1),
            
        nn.Linear(16*5*5, 120),
        nn.BatchNorm1d(120),
        nn.ReLU(),
        nn.Dropout(0.2),
            
        nn.Linear(120,84),
        nn.BatchNorm1d(84),
        nn.ReLU(),
        nn.Dropout(0.2),
        
        nn.Linear(84,10)
         
        )
       
               
    def forward(self, input):
        return self.layers(input)
        
        

In [43]:
model = Net1()
print(model)

Net1(
  (layers): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=400, out_features=120, bias=True)
    (10): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Dropout(p=0.2, inplace=False)
    (13): Linear(in_features=120, out_features=84, bias=True)
    (14): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): ReLU()
    (16): Dropout(p=0.2, inplace=False)
    (17): Linear(

# 3. Define a Loss function and optimizer

In [48]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01,  weight_decay=0.01)
#optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.001) L1 regularization
#optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.001, momentum=0.9) L2 regularization

# 4. Train the network

In [34]:
for epoch in range(2):
    total_loss=0.0
    i=0
    for i, data in enumerate(trainloader,0):
        inputs,labels=data
        
        optimizer.zero_grad()
        
        pred= model(inputs)
        loss=criterion(pred,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() #per epoch
        
    print(f'epoch: {epoch+1} loss:{total_loss/i:.3f}')
            
    
print("Finished training of 50000 images (12500 batches)")
    

epoch: 1 loss:1.909
epoch: 2 loss:1.774
Finished training of 50000 images (12500 batches)


In [35]:
print(i) #batch

12499


# 5. Test the network on the test data

In [36]:
correct=0.0
total=0.0

In [37]:
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs= model(images)
        _, pred= torch.max(outputs.data,1)
        total+= labels.size(0)
        correct += (pred == labels).sum().item()
    print(total)
    print(correct)
print(f'Accuracy of 10000 images(2500 batches): {100 * correct // total} %')        

10000.0
3809.0
Accuracy of 10000 images(2500 batches): 38.0 %


# 6. Save the model

In [38]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)